In [21]:
import pyspark
from operator import add
from pyspark import SparkConf
from pyspark.ml.feature import NGram
from pyspark.sql.functions import col,udf
from pyspark.sql import SQLContext,Row
from operator import add
import numpy as np
import string
from pyspark.ml.feature import HashingTF, IDF, Tokenizer,NGram

#sc = pyspark.SparkContext('local[*]',appName="DocClassification")
#sqlc = SQLContext(sc)



In [22]:
data = sc.parallelize(["This is something I really want to test.",
                       "To test something is something I really want to do.",
                       "I need to do something that I want to do which is to test something.",
                       "This is something I really want to test.",
                       "To test something is something I really want to do.",
                       "I need to do something that I want to do which is to test something.",
                       "This is something I really want to test.",
                       "To test something is something I really want to do.",
                       "I need to do something that I want to do which is to test something.",
                       "This is something I really want to test.",
                       "To test something is something I really want to do.",
                       "I need to do something that I want to do which is to test something."])\
            .zipWithIndex()\
            .toDF(['doc','did'])

data.show()

+--------------------+---+
|                 doc|did|
+--------------------+---+
|This is something...|  0|
|To test something...|  1|
|I need to do some...|  2|
|This is something...|  3|
|To test something...|  4|
|I need to do some...|  5|
|This is something...|  6|
|To test something...|  7|
|I need to do some...|  8|
|This is something...|  9|
|To test something...| 10|
|I need to do some...| 11|
+--------------------+---+



In [23]:
toker = Tokenizer(inputCol = "doc",outputCol = "words")
data = toker.transform(data)
grammer = NGram(n=2,inputCol="words",outputCol="grams")
data = grammer.transform(data)
data.show()

+--------------------+---+--------------------+--------------------+
|                 doc|did|               words|               grams|
+--------------------+---+--------------------+--------------------+
|This is something...|  0|[this, is, someth...|[this is, is some...|
|To test something...|  1|[to, test, someth...|[to test, test so...|
|I need to do some...|  2|[i, need, to, do,...|[i need, need to,...|
|This is something...|  3|[this, is, someth...|[this is, is some...|
|To test something...|  4|[to, test, someth...|[to test, test so...|
|I need to do some...|  5|[i, need, to, do,...|[i need, need to,...|
|This is something...|  6|[this, is, someth...|[this is, is some...|
|To test something...|  7|[to, test, someth...|[to test, test so...|
|I need to do some...|  8|[i, need, to, do,...|[i need, need to,...|
|This is something...|  9|[this, is, someth...|[this is, is some...|
|To test something...| 10|[to, test, someth...|[to test, test so...|
|I need to do some...| 11|[i, need

In [24]:
data = data.rdd.map(lambda x: Row(did=x['did'],features=x['words']+x['grams'])).toDF()
data.show()

+---+--------------------+
|did|            features|
+---+--------------------+
|  0|[this, is, someth...|
|  1|[to, test, someth...|
|  2|[i, need, to, do,...|
|  3|[this, is, someth...|
|  4|[to, test, someth...|
|  5|[i, need, to, do,...|
|  6|[this, is, someth...|
|  7|[to, test, someth...|
|  8|[i, need, to, do,...|
|  9|[this, is, someth...|
| 10|[to, test, someth...|
| 11|[i, need, to, do,...|
+---+--------------------+



You can very easily add the labels onto your DF if you already have them joined in...

In [25]:
hasher = HashingTF(inputCol = 'features',outputCol='rawFeatures')
data = hasher.transform(data)
data.show()

+---+--------------------+--------------------+
|did|            features|         rawFeatures|
+---+--------------------+--------------------+
|  0|[this, is, someth...|(262144,[14,15889...|
|  1|[to, test, someth...|(262144,[14,15889...|
|  2|[i, need, to, do,...|(262144,[4249,158...|
|  3|[this, is, someth...|(262144,[14,15889...|
|  4|[to, test, someth...|(262144,[14,15889...|
|  5|[i, need, to, do,...|(262144,[4249,158...|
|  6|[this, is, someth...|(262144,[14,15889...|
|  7|[to, test, someth...|(262144,[14,15889...|
|  8|[i, need, to, do,...|(262144,[4249,158...|
|  9|[this, is, someth...|(262144,[14,15889...|
| 10|[to, test, someth...|(262144,[14,15889...|
| 11|[i, need, to, do,...|(262144,[4249,158...|
+---+--------------------+--------------------+



In [33]:

len(data.where(data.did==0).select('features').rdd.map(lambda x: x[0]).collect()[0])

15